In [25]:
#pip install legacy-cgi googletrans==3.1.0a0

In [26]:
import os
import pandas as pd
import numpy as np
#from googletrans import Translator

In [27]:
os.listdir()

['.config',
 'incidents-clean.csv',
 'donneesouvertes-interventions-sim2020.csv',
 'donneesouvertes-interventions-sim_2015_2022.csv',
 'donneesouvertes-interventions-sim.csv',
 'donneesouvertes-interventions-sim-2005-2014.csv',
 'sample_data']

In [28]:
df_2005_2014 = pd.read_csv('donneesouvertes-interventions-sim-2005-2014.csv')
df_2015_2022 = pd.read_csv('donneesouvertes-interventions-sim_2015_2022.csv').drop(columns=['MTM8_X', 'MTM8_Y'])
df_2020_2023 = pd.read_csv('donneesouvertes-interventions-sim2020.csv').drop(columns=['MTM8_X', 'MTM8_Y'])
df_2024_2025 = pd.read_csv('donneesouvertes-interventions-sim.csv').drop(columns=['MTM8_X', 'MTM8_Y'])

In [29]:
df_2005_2014['CREATION_DATE_TIME'] = pd.to_datetime(df_2005_2014['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2015_2022['CREATION_DATE_TIME'] = pd.to_datetime(df_2015_2022['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2020_2023['CREATION_DATE_TIME'] = pd.to_datetime(df_2020_2023['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2024_2025['CREATION_DATE_TIME'] = pd.to_datetime(df_2024_2025['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')

In [30]:
df_incidents = pd.concat([df_2005_2014, df_2015_2022, df_2020_2023, df_2024_2025], ignore_index=True)
df_incidents.rename(columns={'CREATION_DATE_TIME': 'CREATION_DATE'}, inplace=True)
df_incidents.drop_duplicates(subset=['INCIDENT_NBR', 'CREATION_DATE'], inplace=True)
df_incidents.shape

(2050879, 11)

In [31]:
df_incidents.sort_values(by=['CREATION_DATE','INCIDENT_NBR'], inplace=True)
df_incidents.head()

,INCIDENT_NBR,CREATION_DATE,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,LATITUDE,LONGITUDE,NOMBRE_UNITES
0,1,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.620274,-73.619956,1.0
1,2,2005-01-01,Alarme privé ou locale,Alarmes-incendies,10,Montréal,Ville-Marie / Parc Jean-Drapeau / Centre-Sud,18,45.494087,-73.582587,7.0
2,3,2005-01-01,Alarme privé ou locale,Alarmes-incendies,72,Saint-Laurent,St-Laurent,13,45.484406,-73.693038,6.0
3,4,2005-01-01,Inondation,Sans incendie,13,Montréal,Mercier / Hochelaga-Maisonneuve,23,45.541383,-73.545944,1.0
4,5,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.611304,-73.632440,1.0


In [32]:
df_incidents['DESCRIPTION_GROUPE'].unique()

array(['Sans incendie', 'Alarmes-incendies', 'Autres incendies',
       'Fausses alertes/annulations', 'Premier répondant',
       'Incendies de bâtiments', 'SANS FEU', '1-REPOND', 'AUTREFEU',
       'INCENDIE', 'FAU-ALER', 'NOUVEAU', nan], dtype=object)

In [33]:
description_groups = {
    "1-REPOND": "Premier répondant",
    "SANS FEU": "Sans incendie",
    "AUTREFEU": "Autres incendies",
    "INCENDIE": "Incendies de bâtiments",
    "FAU-ALER": "Fausses alertes/annulations",
    "Alarmes-incendies": "Alarmes incendies",
    "New": "Noveau"
}
df_incidents = df_incidents.replace({'DESCRIPTION_GROUPE': description_groups})
df_incidents['DESCRIPTION_GROUPE'].value_counts().sort_values(ascending=False)

,count
DESCRIPTION_GROUPE,
Premier répondant,1228687
Sans incendie,434768
Alarmes incendies,286956
Autres incendies,67051
Incendies de bâtiments,26850
Fausses alertes/annulations,5526
NOUVEAU,7


In [34]:
description_groups_translated = {
    "Premier répondant": "First response",
    "Sans incendie": "No fire",
    "Autres incendies": "Other fires",
    "Incendies de bâtiments": "Building fire",
    "Fausses alertes/annulations": "False alarm/cancellation",
    "Alarmes incendies": "Fire Alarm",
    "Noveau": "New"
}
df_incidents['DESC_GROUP_ENGLISH'] = df_incidents['DESCRIPTION_GROUPE'].map(description_groups_translated)
df_incidents['DESC_GROUP_ENGLISH'].value_counts().sort_values(ascending=False)

,count
DESC_GROUP_ENGLISH,
First response,1228687
No fire,434768
Fire Alarm,286956
Other fires,67051
Building fire,26850
False alarm/cancellation,5526


In [35]:
"""
translator = Translator()
df_type_descriptions = pd.read_csv('type-interventions-descriptions20161122.csv')
df_type_descriptions.rename(columns={'INCIDENT_TYPE_DESCRIPTION': 'INCIDENT_TYPE_DESC', 'Description': 'DESCRIPTION'}, inplace=True)
df_type_descriptions.drop(
    df_type_descriptions[df_type_descriptions['DESCRIPTION'] == "Type qui n'est plus utilisé à la suite de la mise à jour de l'algorithme en 2016"].index,
    inplace=True
)
df_type_descriptions['DESC_ENGLISH'] = df_type_descriptions['DESCRIPTION'].apply(translator.translate, src='fr', dest='en').apply(getattr, args=('text',))
df_type_descriptions.head()
"""

'\ntranslator = Translator()\ndf_type_descriptions = pd.read_csv(\'type-interventions-descriptions20161122.csv\')\ndf_type_descriptions.rename(columns={\'INCIDENT_TYPE_DESCRIPTION\': \'INCIDENT_TYPE_DESC\', \'Description\': \'DESCRIPTION\'}, inplace=True)\ndf_type_descriptions.drop(\n    df_type_descriptions[df_type_descriptions[\'DESCRIPTION\'] == "Type qui n\'est plus utilisé à la suite de la mise à jour de l\'algorithme en 2016"].index,\n    inplace=True\n)\ndf_type_descriptions[\'DESC_ENGLISH\'] = df_type_descriptions[\'DESCRIPTION\'].apply(translator.translate, src=\'fr\', dest=\'en\').apply(getattr, args=(\'text\',))\ndf_type_descriptions.head()\n'

In [36]:
"""
df_merged = pd.merge(df_incidents, df_type_descriptions, on='INCIDENT_TYPE_DESC', how='left')
df_merged = df_merged.iloc[:, [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13]]
print(df_merged.shape)
df_merged.head()
"""

"\ndf_merged = pd.merge(df_incidents, df_type_descriptions, on='INCIDENT_TYPE_DESC', how='left')\ndf_merged = df_merged.iloc[:, [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13]]\nprint(df_merged.shape)\ndf_merged.head()\n"

In [37]:
df_na = df_incidents[df_incidents.isna().any(axis=1) == True]
df_na.shape

(1791, 12)

In [38]:
df_na.isna().sum()

,0
INCIDENT_NBR,0
CREATION_DATE,0
INCIDENT_TYPE_DESC,0
DESCRIPTION_GROUPE,1034
CASERNE,0
NOM_VILLE,0
NOM_ARROND,0
DIVISION,0
LATITUDE,1
LONGITUDE,1


In [39]:
df_incidents.dropna(axis=0, inplace=True)
df_incidents.shape

(2049088, 12)

In [40]:
df_incidents['DESC_GROUP_ENGLISH'].value_counts()

,count
DESC_GROUP_ENGLISH,
First response,1227941
No fire,434764
Fire Alarm,286956
Other fires,67051
Building fire,26850
False alarm/cancellation,5526


In [41]:
df_incidents = df_incidents[df_incidents['DESC_GROUP_ENGLISH'].isin(['Building fire', 'Other fires'])]
df_incidents.shape
df_incidents['DESC_GROUP_ENGLISH'].value_counts()

,count
DESC_GROUP_ENGLISH,
Other fires,67051
Building fire,26850


In [42]:
np.sort(df_incidents['CASERNE'].unique())

array([ 3,  4,  5,  8,  9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42,
       43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
       61, 62, 63, 64, 65, 66, 67, 71, 72, 73, 74, 75, 76, 77, 78, 79])

In [43]:
 nom_villes_replace = {
    "Baie d'Urfé": "Baie-d'Urfé",
    "Côte St-Luc" : "Côte-Saint-Luc",
    "Sainte-Anne-de-Bellevue" : "Ste-Anne-de-Bellevue"
}
df_incidents = df_incidents.replace({'NOM_VILLE': nom_villes_replace})
df_incidents['NOM_VILLE'].value_counts().sort_index(ascending=True)

,count
NOM_VILLE,
Anjou,319
Baie-d'Urfé,216
Beaconsfield,506
Côte-Saint-Luc,936
Dollard-des-Ormeaux,1415
Dorval,1359
Hampstead,176
Ile-Bizard,101
Kirkland,630


In [44]:
 nom_arrond_replace = {
    "Ahuntsic-Cartierville": "Ahuntsic / Cartierville",
    "Côte-des-Neiges-Notre-Dame-de-Grâce" : "Côte-des-Neiges / Notre-Dame-de-Grâce",
    "L'Île-Bizard-Sainte-Geneviève" : "L'Ile-Bizard / Ste-Geneviève",
    "Lasalle" : "LaSalle",
    "Le Plateau-Mont-Royal": "Mont-Royal",
    "Plateau Mont-Royal": "Mont-Royal",
    "Le Sud-Ouest": "Sud-Ouest",
    "Mercier-Hochelaga-Maisonneuve": "Mercier / Hochelaga-Maisonneuve",
    "Pierrefonds-Roxboro": "Pierrefonds / Roxboro",
    "Rivière-des-Prairies / P-A-T/Mtl-Est": "Rivière-des-Prairies / Pointe-aux-Trembles",
    "Rivière-des-Prairies-Pointe-aux-Trembles": "Rivière-des-Prairies / Pointe-aux-Trembles",
    "Rosemont-La Petite-Patrie": "Rosemont / Petite-Patrie",
    "St-Laurent": "Saint-Laurent",
    "St-Léonard": "Saint-Léonard",
    "Verdun / Ïle-des-Soeurs": "Verdun",
    "Ville-Marie / Parc Jean-Drapeau / Centre-Sud": "Ville-Marie",
    "Villeray-Saint-Michel-Parc-Extension": "Villeray / St-Michel / Parc Extension",
}
df_incidents = df_incidents.replace({'NOM_ARROND': nom_arrond_replace})
df_incidents['NOM_ARROND'].value_counts().sort_index(ascending=True)

,count
NOM_ARROND,
Ahuntsic / Cartierville,5423
Anjou,1697
Beaconsfield / Baie d'Urfé,161
Côte St-Luc / Hampstead / Mtl-Ouest,222
Côte-des-Neiges / Notre-Dame-de-Grâce,6235
Dollard-des-Ormeaux / Roxboro,341
Dorval / Ile Dorval,256
Ile-Bizard / Ste-Geneviève / Ste-A-de-B,197
Indéterminé,7544


In [45]:
np.sort(df_incidents['DIVISION'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17,
       18, 20, 21, 22, 23, 25])

In [46]:
np.sort(df_incidents['NOMBRE_UNITES'].unique())

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  98., 100., 101.,
       102., 103., 104., 105., 106., 107., 108., 115., 117., 118., 119.,
       120., 123., 126., 127., 130., 132., 135., 148., 153., 160., 163.,
       172., 199., 244., 275.])

In [47]:
df_incidents = df_incidents[pd.to_datetime(df_incidents['CREATION_DATE']).dt.year < 2025]
df_incidents.shape
np.sort(pd.to_datetime(df_incidents['CREATION_DATE']).dt.year.unique())


array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], dtype=int32)

In [48]:
df_incidents.to_csv('incidents-clean.csv', index=False)